In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_csv('Resources/spam.csv', encoding = 'latin-1')
data1.head()

,label,msg,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
list = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
data1 = data1.drop(list, axis=1)

In [4]:
data2 = pd.ExcelFile('Resources/indiandataset1.xls')
data2 = data2.parse(0)
data2.head()

,code,label,msg
0,1,ham,"Dear Customer, +916300623587 is now available ..."
1,1,ham,"Dear Customer, You have a missed call from +91..."
2,1,spam,"Join Hike to get Rs 40. Earn upto Rs. 10,000 b..."
3,1,spam,Just sent you some money and invited you to Hi...
4,1,spam,Just sent you some money and invited you to Hi...


# for english ...

In [5]:
lst = []
i = 0
for row in data2.itertuples():
    if not int(row[1]) == 1:
        lst.append(i)
    i = i + 1

In [6]:
print(len(lst))

850


In [7]:
data2_eng = data2.drop(data2.index[lst], axis = 0) # dropped non engish rows

In [8]:
data_eng = pd.concat([data1, data2_eng])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data_eng['msg'], data_eng['label'], test_size = 0.30)

Y_train = pd.get_dummies(Y_train).values

Text processing   ->  __splitting, removing punctuation, lowering (Lower Casing)__

In [10]:
from keras.preprocessing.text import text_to_word_sequence
temp = []
for i in range(len(X_train)):
    words = []
    rev = str(X_train.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
X_train = temp

/home/manohar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Converting Words to Vectors

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 3000

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train)

RNN model building

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

embed_dim = X_train.shape[1]
lstm_out = 196


model = Sequential()
model.add(
    Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/home/manohar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 175, 175)          525000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               291648    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 817,042
Trainable params: 817,042
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(X_train, Y_train, epochs = 5,batch_size = 20)

Epoch 1/5
6521/6521 [==============================] - 99s 15ms/step - loss: 0.1699 - acc: 0.9368
Epoch 2/5
6521/6521 [==============================] - 89s 14ms/step - loss: 0.0601 - acc: 0.9802
Epoch 3/5
6521/6521 [==============================] - 85s 13ms/step - loss: 0.0367 - acc: 0.9880
Epoch 4/5
6521/6521 [==============================] - 83s 13ms/step - loss: 0.0339 - acc: 0.9893
Epoch 5/5
6521/6521 [==============================] - 95s 15ms/step - loss: 0.0202 - acc: 0.9943


Training set text preprocessing

In [14]:
temp = []
for i in range(len(X_test)):
    words = []
    rev = str(X_test.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
    
temp = tokenizer.texts_to_sequences(temp)
le = X_train.shape[1]
temp2 = [[0 for i in range(le)] for j in range(len(temp))]

for i in range(len(temp)):
    for j in range(len(temp[i])-1,-1,-1):
        temp2[i][le-1+j-len(temp[i])+1] = temp[i][j];

temp2 = np.array(temp2).reshape(len(temp),le,)

In [15]:
Y_pr = model.predict(temp2)

Y_pred = []

for i in range(len(Y_pr)):
    if Y_pr[i][0] >= Y_pr[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')
        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)  
print(cm)

[[2219   39]
 [  55  483]]


Wrongly Predicted to a file

In [16]:
import csv
file = open('output/wrongly_predicted_eng.csv', 'w')
wr_file = csv.writer(file)
wr_file.writerow(["label","msg"])

11

In [17]:
for i in range(len(Y_test)):    
    if not Y_test.iloc[i] == Y_pred[i]:
        wr_file.writerow([Y_test.iloc[i],X_test.iloc[i]])
file.close()

# For Hindi ..

In [18]:
lst = []
i = 0
for row in data2.itertuples():
    if not int(row[1]) == 2:
        lst.append(i)
    i = i + 1

In [19]:
data_hin = data2.drop(data2.index[lst], axis = 0)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data_hin['msg'], data_hin['label'], test_size = 0.30)

Y_train = pd.get_dummies(Y_train).values

In [21]:
from keras.preprocessing.text import text_to_word_sequence
temp = []
for i in range(len(X_train)):
    words = []
    rev = str(X_train.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
X_train = temp

In [22]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 600

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train)

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

embed_dim = X_train.shape[1]
lstm_out = 196


model = Sequential()
model.add(
    Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/home/manohar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35, 35)            21000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               181888    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 203,282
Trainable params: 203,282
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
model.fit(X_train, Y_train, epochs = 12,batch_size = 20)

Epoch 1/12
107/107 [==============================] - 1s 10ms/step - loss: 0.6723 - acc: 0.7850
Epoch 2/12
107/107 [==============================] - 0s 3ms/step - loss: 0.5133 - acc: 0.8131
Epoch 3/12
107/107 [==============================] - 0s 4ms/step - loss: 0.4322 - acc: 0.8131
Epoch 4/12
107/107 [==============================] - 0s 4ms/step - loss: 0.3820 - acc: 0.8131
Epoch 5/12
107/107 [==============================] - 0s 4ms/step - loss: 0.3051 - acc: 0.8692
Epoch 6/12
107/107 [==============================] - 0s 4ms/step - loss: 0.2069 - acc: 0.9626
Epoch 7/12
107/107 [==============================] - 0s 4ms/step - loss: 0.1320 - acc: 0.9626
Epoch 8/12
107/107 [==============================] - 0s 3ms/step - loss: 0.0969 - acc: 0.9720
Epoch 9/12
107/107 [==============================] - 0s 3ms/step - loss: 0.0827 - acc: 0.9813
Epoch 10/12
107/107 [==============================] - 0s 3ms/step - loss: 0.0551 - acc: 1.0000
Epoch 11/12
107/107 [===========================

In [25]:
temp = []
for i in range(len(X_test)):
    words = []
    rev = str(X_test.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
    
temp = tokenizer.texts_to_sequences(temp)
le = X_train.shape[1]
temp2 = [[0 for i in range(le)] for j in range(len(temp))]

for i in range(len(temp)):
    for j in range(len(temp[i])-1,-1,-1):
        temp2[i][le-1+j-len(temp[i])+1] = temp[i][j];

temp2 = np.array(temp2).reshape(len(temp),le,)

In [26]:
Y_pr = model.predict(temp2)

Y_pred = []

for i in range(len(Y_pr)):
    if Y_pr[i][0] >= Y_pr[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')
        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)  
print(cm)

[[10  1]
 [ 1 35]]


Wrongly predicted to File

In [27]:
import csv
file = open('output/wrongly_predicted_hin.csv', 'w')
wr_file = csv.writer(file)
wr_file.writerow(["label","msg"])

11

In [28]:
for i in range(len(Y_test)):    
    if not Y_test.iloc[i] == Y_pred[i]:
        wr_file.writerow([Y_test.iloc[i],X_test.iloc[i]])
file.close()

# for Code Mixing

In [29]:
lst = []
i = 0
for row in data2.itertuples():
    if not int(row[1]) == 3:
        lst.append(i)
    i = i + 1

In [30]:
data_mix = data2.drop(data2.index[lst], axis = 0)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data_mix['msg'], data_mix['label'], test_size = 0.30)

Y_train = pd.get_dummies(Y_train).values

In [32]:
from keras.preprocessing.text import text_to_word_sequence
temp = []
for i in range(len(X_train)):
    words = []
    rev = str(X_train.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
X_train = temp

In [33]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 600

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

embed_dim = X_train.shape[1]
lstm_out = 196


model = Sequential()
model.add(
    Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/home/manohar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 35, 35)            21000     
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               181888    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 203,282
Trainable params: 203,282
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
model.fit(X_train, Y_train, epochs = 12,batch_size = 20)

Epoch 1/12
122/122 [==============================] - 1s 10ms/step - loss: 0.6688 - acc: 0.8852
Epoch 2/12
122/122 [==============================] - 1s 5ms/step - loss: 0.5652 - acc: 0.8443
Epoch 3/12
122/122 [==============================] - 0s 3ms/step - loss: 0.4629 - acc: 0.9098
Epoch 4/12
122/122 [==============================] - 0s 3ms/step - loss: 0.4742 - acc: 0.8443
Epoch 5/12
122/122 [==============================] - 0s 3ms/step - loss: 0.3205 - acc: 0.9754
Epoch 6/12
122/122 [==============================] - 0s 3ms/step - loss: 0.2174 - acc: 0.9426
Epoch 7/12
122/122 [==============================] - 0s 4ms/step - loss: 0.1171 - acc: 0.9836
Epoch 8/12
122/122 [==============================] - 1s 5ms/step - loss: 0.1003 - acc: 1.0000
Epoch 9/12
122/122 [==============================] - 1s 5ms/step - loss: 0.0422 - acc: 0.9918
Epoch 10/12
122/122 [==============================] - 1s 5ms/step - loss: 0.0445 - acc: 0.9836
Epoch 11/12
122/122 [===========================

In [36]:
temp = []
for i in range(len(X_test)):
    words = []
    rev = str(X_test.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
    
temp = tokenizer.texts_to_sequences(temp)
le = X_train.shape[1]
temp2 = [[0 for i in range(le)] for j in range(len(temp))]

for i in range(len(temp)):
    for j in range(len(temp[i])-1,-1,-1):
        temp2[i][le-1+j-len(temp[i])+1] = temp[i][j];

temp2 = np.array(temp2).reshape(len(temp),le,)

In [37]:
Y_pr = model.predict(temp2)

Y_pred = []

for i in range(len(Y_pr)):
    if Y_pr[i][0] >= Y_pr[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')
        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)  
print(cm)

[[32  0]
 [ 1 20]]


Wrongly predicted to File

In [38]:
import csv
file = open('output/wrongly_predicted_mix.csv', 'w')
wr_file = csv.writer(file)
wr_file.writerow(["label","msg"])

11

In [39]:
for i in range(len(Y_test)):    
    if not Y_test.iloc[i] == Y_pred[i]:
        wr_file.writerow([Y_test.iloc[i],X_test.iloc[i]])
file.close()

# Code Mixing

In [49]:
lst = []
i = 0
for row in data2.itertuples():
    if not int(row[1]) == 4:
        lst.append(i)
    i = i + 1

In [50]:
data_tel = data2.drop(data2.index[lst], axis = 0)

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data_tel['msg'], data_tel['label'], test_size = 0.30)

Y_train = pd.get_dummies(Y_train).values

In [52]:
from keras.preprocessing.text import text_to_word_sequence
temp = []
for i in range(len(X_train)):
    words = []
    rev = str(X_train.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
X_train = temp

In [53]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train)

In [54]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

embed_dim = X_train.shape[1]
lstm_out = 196


model = Sequential()
model.add(
    Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/home/manohar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 69, 69)            13800     
_________________________________________________________________
lstm_5 (LSTM)                (None, 196)               208544    
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 394       
Total params: 222,738
Trainable params: 222,738
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
model.fit(X_train, Y_train, epochs = 10,batch_size = 10)

Epoch 1/10
364/364 [==============================] - 6s 17ms/step - loss: 0.3489 - acc: 0.9560
Epoch 2/10
364/364 [==============================] - 5s 13ms/step - loss: 0.1381 - acc: 0.9698
Epoch 3/10
364/364 [==============================] - 5s 13ms/step - loss: 0.1340 - acc: 0.9698
Epoch 4/10
364/364 [==============================] - 5s 14ms/step - loss: 0.1199 - acc: 0.9698
Epoch 5/10
364/364 [==============================] - 5s 14ms/step - loss: 0.1086 - acc: 0.9698
Epoch 6/10
364/364 [==============================] - 5s 13ms/step - loss: 0.0939 - acc: 0.9698
Epoch 7/10
364/364 [==============================] - 5s 14ms/step - loss: 0.0696 - acc: 0.9808
Epoch 8/10
364/364 [==============================] - 5s 14ms/step - loss: 0.0626 - acc: 0.9808
Epoch 9/10
364/364 [==============================] - 5s 13ms/step - loss: 0.0579 - acc: 0.9863
Epoch 10/10
364/364 [==============================] - 5s 12ms/step - loss: 0.0599 - acc: 0.9835


In [59]:
temp = []
for i in range(len(X_test)):
    words = []
    rev = str(X_test.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
    
temp = tokenizer.texts_to_sequences(temp)
le = X_train.shape[1]
temp2 = [[0 for i in range(le)] for j in range(len(temp))]

for i in range(len(temp)):
    for j in range(len(temp[i])-1,-1,-1):
        temp2[i][le-1+j-len(temp[i])+1] = temp[i][j];

temp2 = np.array(temp2).reshape(len(temp),le,)

In [60]:
Y_pr = model.predict(temp2)

Y_pred = []

for i in range(len(Y_pr)):
    if Y_pr[i][0] >= Y_pr[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')
        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)  
print(cm)

[[152   0]
 [  3   1]]


wrongly predicted to file

In [61]:
import csv
file = open('output/wrongly_predicted_tel.csv', 'w')
wr_file = csv.writer(file)
wr_file.writerow(["label","msg"])

11

In [62]:
for i in range(len(Y_test)):    
    if not Y_test.iloc[i] == Y_pred[i]:
        wr_file.writerow([Y_test.iloc[i],X_test.iloc[i]])
file.close()